# Agent 2 — Module Walkthrough (Code + Review)
## Time-Series Classifier (`model.py`)

**Author:** Summer Xiong  
**Goal:** Explain the architecture and forward pass of the Agent 2 model, including tensor shapes and design rationale.

This module defines `TimeSeriesClassifier`, a hybrid model that combines:
1. **Pretrained text encoder** (RoBERTa via `AutoModel`) applied per step  
2. **Numeric feature projection** applied per step  
3. **Temporal Transformer encoder** over the window dimension `W`  
4. A **classification head** that predicts the current-step vote label (For/Against/Abstain)

> **Key idea:** Each training sample is a window of `W` steps. Each step has:
> - proposal text (tokenised)
> - numeric features (vp, vp_share, time features, etc.)
> The model encodes each step and then models temporal dependencies across steps.


## 0) Imports

- `torch`, `torch.nn`: model components and training
- `transformers.AutoModel`: load a pretrained transformer encoder (e.g., RoBERTa)


In [ ]:
from typing import Dict, Optional

import torch
import torch.nn as nn
from transformers import AutoModel


## 1) Class Overview: `TimeSeriesClassifier`

### Constructor parameters (what they control)
- `pretrained_model_name`: text encoder backbone (default: `roberta-base`)
- `hidden_dim`: projected text representation size per step
- `feat_dim`: numeric feature dimension per step (must match your window builder output)
- `label_emb_dim`: embedding dim for label tokens (see review note below)
- `pos_emb_dim`: embedding dim for step positions in the window
- `num_heads`, `ff_dim`: Transformer encoder hyperparameters
- `num_classes`: 3 by default (For/Against/Abstain)
- `dropout`: regularisation

### High-level computation graph
For each batch:
1. Encode each step text with RoBERTa → take `[CLS]` token representation  
2. Project numeric features through an MLP  
3. Add position embedding (and optionally label embedding)  
4. Concatenate all step features → run temporal Transformer over steps  
5. Take last step representation → classify into 3 classes


In [ ]:
class TimeSeriesClassifier(nn.Module):
    def __init__(self,
                 pretrained_model_name: str = "roberta-base",
                 hidden_dim: int = 256,
                 feat_dim: int = 8,
                 label_emb_dim: int = 128,
                 pos_emb_dim: int = 128,
                 num_heads: int = 8,
                 ff_dim: int = 1024,
                 num_classes: int = 3,
                 dropout: float = 0.1):
        super().__init__()
        self.text_encoder = AutoModel.from_pretrained(pretrained_model_name)
        self.hidden_dim = hidden_dim
        self.text_proj = nn.Linear(self.text_encoder.config.hidden_size, hidden_dim)
        self.num_classes = num_classes

        # label/position embeddings
        self.label_emb = nn.Embedding(num_classes + 1, label_emb_dim)  # +1 for current-step placeholder
        self.pos_emb = nn.Embedding(512, pos_emb_dim)                  # support up to W<=512

        # numeric projection
        self.feat_proj = nn.Sequential(
            nn.Linear(feat_dim, 128),
            nn.ReLU(),
            nn.LayerNorm(128),
            nn.Dropout(dropout),
            nn.Linear(128, 128),
        )

        fused_dim = hidden_dim + label_emb_dim + pos_emb_dim + 128

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=fused_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        self.temporal = nn.TransformerEncoder(encoder_layer, num_layers=2)

        self.cls_head = nn.Sequential(
            nn.Linear(fused_dim, 256),
            nn.ReLU(),
            nn.LayerNorm(256),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )

        # temperature (optional calibration)
        self.temperature = nn.Parameter(torch.ones(1))


## 2) Component-by-Component Explanation

### 2.1 Text encoder (`self.text_encoder`)
A pretrained transformer (RoBERTa by default) outputs contextual token embeddings.

You extract the representation of the first token `[:, 0, :]` (often treated as a sentence embedding):
- shape per step: `(H_text,)`
- `H_text = self.text_encoder.config.hidden_size`

Then project it to `hidden_dim` using `self.text_proj` to:
- reduce dimensionality
- control the fused representation size

---

### 2.2 Numeric feature projection (`self.feat_proj`)
`feat_proj` maps per-step numeric features of size `feat_dim` into a 128-dimensional embedding.

This makes numeric features comparable in scale/expressiveness to text embeddings.

**Important coupling:** `feat_dim` must exactly match the number of per-step numeric features produced by your window builder.

---

### 2.3 Position embedding (`self.pos_emb`)
Encodes step index `0..W-1` into `pos_emb_dim`.

This gives the temporal transformer awareness of step order.

Note: you hardcode support for `W<=512`. If your window size is always small (e.g., 5–50), this is fine.

---

### 2.4 Label embedding (`self.label_emb`)
This is defined, but in `forward()` you currently set all label tokens to zero.  
So *as implemented*, it is effectively a constant vector added to every step.

This is not harmful, but it is not providing meaningful information unless you supply step-level label ids.

See review notes in Section 4.


## 3) Forward Pass (Step-by-step with Shapes)

The `batch` dictionary is expected to contain:
- `input_ids`: `(B, W, L)`
- `attention_mask`: `(B, W, L)`
- `num_feats`: `(B, W, F)` where `F = feat_dim`
- `labels`: `(B,)`
- `clusters`: `(B,)`

The model returns:
- `logits`: `(B, C)` where `C=num_classes`


In [ ]:
def forward(self, batch: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    """
    batch: input_ids (B,W,L), attention_mask (B,W,L), num_feats (B,W,F), labels (B,), clusters (B,)
    """
    input_ids = batch["input_ids"]      # (B,W,L)
    attn = batch["attention_mask"]      # (B,W,L)
    feats = batch["num_feats"]          # (B,W,F)
    B, W, L = input_ids.size()

    # flatten steps for text encoding
    x = input_ids.view(B * W, L)
    a = attn.view(B * W, L)
    enc = self.text_encoder(input_ids=x, attention_mask=a)

    cls = enc.last_hidden_state[:, 0, :]              # (B*W, Htext)
    cls = self.text_proj(cls)                         # (B*W, hidden_dim)
    cls = cls.view(B, W, -1)                          # (B, W, hidden_dim)

    # label & position embeddings
    pos_ids = torch.arange(W, device=input_ids.device).unsqueeze(0).expand(B, W)
    pos = self.pos_emb(pos_ids)                       # (B, W, pos_emb_dim)

    # label embeddings (currently all zeros)
    label_tokens = torch.zeros((B, W), dtype=torch.long, device=input_ids.device)
    lab = self.label_emb(label_tokens)                # (B, W, label_emb_dim)

    # numeric projection
    fproj = self.feat_proj(feats)                     # (B, W, 128)

    # fuse
    fused = torch.cat([cls, lab, pos, fproj], dim=-1) # (B, W, fused_dim)

    # temporal modelling over steps
    seq = self.temporal(fused)                        # (B, W, fused_dim)

    # classify using the last step representation (current step)
    last = seq[:, -1, :]                              # (B, fused_dim)
    logits = self.cls_head(last)                      # (B, num_classes)

    return {"logits": logits}


### 3.1 Why flatten `B*W` for text encoding?

The pretrained text encoder expects shape `(batch, seq_len)`.  
Your data has an extra window dimension `(B, W, L)`, so you reshape to:
- `(B*W, L)` to encode all steps in one pass

Then you reshape back to `(B, W, hidden_dim)`.

This is efficient and correct.


## 4) Loss Function and Calibration

### 4.1 `loss_fn`
Uses cross-entropy loss, optionally with `class_weights` to address class imbalance.

### 4.2 `calibrate_logits`
Implements **temperature scaling**: `logits / T`.

This is typically used post-training for calibration (ECE improvement) but can also be learned jointly.

Important: you are returning raw logits in `forward`; calibration is applied externally.


In [ ]:
def loss_fn(self, logits: torch.Tensor, labels: torch.Tensor, class_weights: Optional[torch.Tensor] = None):
    if class_weights is not None:
        loss = nn.CrossEntropyLoss(weight=class_weights)(logits, labels)
    else:
        loss = nn.CrossEntropyLoss()(logits, labels)
    return loss

def calibrate_logits(self, logits: torch.Tensor) -> torch.Tensor:
    # temperature scaling
    return logits / self.temperature.clamp(min=1e-3)


## 5) Review Notes (Strengths, Risks, and Improvements)

### ✅ Strengths
- Clear separation of modalities: text vs numeric
- Good use of pretrained encoder for proposal semantics
- Temporal transformer models cross-step dependencies
- Uses last-step representation to predict current vote (aligned with window design)
- Includes an explicit calibration parameter (temperature)

### ⚠️ Key risks / potential issues
1) **Label embedding is currently unused**
   - In `forward()`, `label_tokens` is all zeros for all steps.
   - That means `lab` is constant, adding no information.

   **If you intended to use history labels**, you have two options:
   - Provide per-step label ids as part of the batch (e.g., from window builder), and set:
     - history steps: true label ids (0/1/2)
     - current step: placeholder id = `num_classes` (the extra +1)
   - Or remove label embeddings entirely to simplify.

2) **`feat_dim` must match actual numeric vector length**
   - Your sliding window builder creates feature vectors:
     - `len(numeric_cols)` + `(#boolean cols included)` + `4 time features`
   - This must exactly equal `feat_dim`, otherwise training will crash.

   **Recommendation:** assert `feats.size(-1) == feat_dim` at runtime.

3) **Position embedding max length**
   - `self.pos_emb = nn.Embedding(512, ...)` supports `W<=512`.
   - If you ever increase `window_size` beyond 512, this will break.

4) **CLS pooling assumption**
   - You use `last_hidden_state[:, 0, :]` as sentence embedding.
   - Works for RoBERTa, but you may also consider mean pooling for robustness.

5) **Calibration usage**
   - Temperature scaling is usually fitted on validation set after training.
   - If you train `temperature` jointly, ensure you are not leaking validation info.

---

### Suggested short-term improvements (publication-ready)
- Implement history label ids properly *or* remove label_emb
- Add assertions for shapes:
  - `input_ids.shape == (B, W, L)`
  - `num_feats.shape[-1] == feat_dim`
- Log parameter choices and report ablations:
  - with/without label prefixes in text
  - with/without explicit label embeddings
  - with/without numeric features


## 6) Minimal Shape Sanity Check (No Transformers Download)

Because `AutoModel.from_pretrained(...)` downloads weights, this notebook provides a **non-downloading shape test** pattern.

In your real training environment, you would instantiate:
```python
model = TimeSeriesClassifier(pretrained_model_name="roberta-base", feat_dim=F)
```

Below we define a tiny dummy module to test tensor flow without external downloads.


In [ ]:
class DummyTextEncoder(nn.Module):
    def __init__(self, hidden_size=32):
        super().__init__()
        self.config = type("cfg", (), {"hidden_size": hidden_size})()
        self.emb = nn.Embedding(1000, hidden_size)

    def forward(self, input_ids, attention_mask=None):
        # input_ids: (N, L)
        x = self.emb(input_ids)  # (N, L, H)
        return type("out", (), {"last_hidden_state": x})()

class TimeSeriesClassifier_NoDownload(TimeSeriesClassifier):
    def __init__(self, **kwargs):
        super(nn.Module, self).__init__()
        # mimic original init without downloading
        pretrained_model_name = kwargs.get("pretrained_model_name", "dummy")
        hidden_dim = kwargs.get("hidden_dim", 16)
        feat_dim = kwargs.get("feat_dim", 8)
        label_emb_dim = kwargs.get("label_emb_dim", 8)
        pos_emb_dim = kwargs.get("pos_emb_dim", 8)
        num_heads = kwargs.get("num_heads", 2)
        ff_dim = kwargs.get("ff_dim", 64)
        num_classes = kwargs.get("num_classes", 3)
        dropout = kwargs.get("dropout", 0.1)

        self.text_encoder = DummyTextEncoder(hidden_size=32)
        self.hidden_dim = hidden_dim
        self.text_proj = nn.Linear(self.text_encoder.config.hidden_size, hidden_dim)
        self.num_classes = num_classes

        self.label_emb = nn.Embedding(num_classes + 1, label_emb_dim)
        self.pos_emb = nn.Embedding(512, pos_emb_dim)

        self.feat_proj = nn.Sequential(
            nn.Linear(feat_dim, 128),
            nn.ReLU(),
            nn.LayerNorm(128),
            nn.Dropout(dropout),
            nn.Linear(128, 128),
        )

        fused_dim = hidden_dim + label_emb_dim + pos_emb_dim + 128

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=fused_dim, nhead=num_heads, dim_feedforward=ff_dim, dropout=dropout, batch_first=True
        )
        self.temporal = nn.TransformerEncoder(encoder_layer, num_layers=2)

        self.cls_head = nn.Sequential(
            nn.Linear(fused_dim, 256),
            nn.ReLU(),
            nn.LayerNorm(256),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes),
        )

        self.temperature = nn.Parameter(torch.ones(1))

    def forward(self, batch: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # paste the original forward logic
        input_ids = batch["input_ids"]
        attn = batch["attention_mask"]
        feats = batch["num_feats"]
        B, W, L = input_ids.size()

        x = input_ids.view(B * W, L)
        a = attn.view(B * W, L)
        enc = self.text_encoder(input_ids=x, attention_mask=a)

        cls = enc.last_hidden_state[:, 0, :]
        cls = self.text_proj(cls)
        cls = cls.view(B, W, -1)

        pos_ids = torch.arange(W, device=input_ids.device).unsqueeze(0).expand(B, W)
        pos = self.pos_emb(pos_ids)

        label_tokens = torch.zeros((B, W), dtype=torch.long, device=input_ids.device)
        lab = self.label_emb(label_tokens)

        fproj = self.feat_proj(feats)

        fused = torch.cat([cls, lab, pos, fproj], dim=-1)
        seq = self.temporal(fused)
        last = seq[:, -1, :]
        logits = self.cls_head(last)
        return {"logits": logits}

# Dummy batch
B, W, L, F = 2, 3, 8, 8
batch = {
    "input_ids": torch.randint(0, 1000, (B, W, L)),
    "attention_mask": torch.ones((B, W, L), dtype=torch.long),
    "num_feats": torch.randn((B, W, F)),
    "labels": torch.randint(0, 3, (B,)),
    "clusters": torch.randint(0, 3, (B,)),
}

m = TimeSeriesClassifier_NoDownload(feat_dim=F)
out = m(batch)
out["logits"].shape


## 7) Summary

`TimeSeriesClassifier` is a multimodal temporal model:
- **RoBERTa** encodes proposal texts per step
- an MLP embeds numeric signals per step
- a **TransformerEncoder** models temporal dependencies across the window
- a classification head predicts the current vote (3-class)

Key implementation checkpoint: ensure `feat_dim` matches your window feature vector length, and decide whether to properly use history labels in `label_emb` or remove it for simplicity.
